# Competitors Dataset

This notebook will unpack the seller's feature and create a new DF where we can see, for each product, which competitors seel it, and how that price relates to ours. There is option to export and import the data to csv and pickle formats after all columns are calculated.


# Imports

In [ ]:
import pandas as pd
import numpy as np
import warnings
import json
import pickle
from collections import Counter
from google.colab import drive

In [ ]:
# import case we need to load something to/from my google drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/walmart c1 bootcamp/df_after_cleaning.csv',
                 usecols=['product_name', 'manufacturer', 'price', 'sellers',
                          'category', 'sub_category1', 'sub_category2',
                          'sub_category3', 'pi_sub_category1', 'pi_sub_category2',
                          'pi_sub_category3'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## <span style="color:red">***sellers***</span>

In [ ]:
# how does seller column look like:
df['sellers'][5622]

"{'seller': [{'Seller_name_1': 'Willis & Simpson', 'Seller_price_1': '£4.99'}, {'Seller_name_2': 'SMM Toys', 'Seller_price_2': '£5.99'}, {'Seller_name_3': 'Otherrealmscollectibles', 'Seller_price_3': '£10.99'}, {'Seller_name_4': 'North East Stock Clearance Supplies', 'Seller_price_4': '£8.95'}, {'Seller_name_5': 'End of Lines North East Ltd', 'Seller_price_5': '£9.99'}]}"

In [ ]:
# let's start by counting competitors:
def count_competitors(df,competitors):
    try:
        if type(eval(df['sellers'])['seller']) == dict: # when sellers >= 2
            key = list(eval(df['sellers'])['seller'].values())[0]
            competitors.append(key)
        elif type(eval(df['sellers'])['seller']) == list: # when sellers = 1
            for i in range(len(list(eval(df['sellers'])['seller']))):
                key = list(eval(df['sellers'])['seller'][i].values())[0]
                competitors.append(key)
    except: pass
    
competitors = []
df.apply(lambda x: count_competitors(x,competitors), axis=1)
comp_count = Counter(competitors)

print('count of competitors', len(comp_count), '\n')
print('top 10 competitors:')
for i in range(11):
    print(comp_count.most_common()[i])

print('\n5th perc.:', comp_count.most_common()[int(len(comp_count)/20)], '\n')
print('10th perc.:', comp_count.most_common()[int(len(comp_count)/5)], '\n')
print('25th perc.:', comp_count.most_common()[int(len(comp_count)/4)], '\n')
print('median:', comp_count.most_common()[int(len(comp_count)/2)])

count of competitors 4137 

top 10 competitors:
('OnlineRetail.co.uk', 1328)
('Buy-For-Less-Online', 473)
('M&E Store-UK', 426)
('M and N Media US', 388)
('MEGA-MAG', 344)
('francejouet', 327)
('Gameseek', 278)
('Sam Store US', 251)
('Gaugemaster', 204)
('Smaller World Future', 200)
('GamesQuest', 186)

5th perc.: ('mikokoro web', 29) 

10th perc.: ('idee+spiel Große', 7) 

25th perc.: ('Jocando', 6) 

median: ('ZeitGeistErleben', 2)


The "competitors" that appears as OnlineRetail.co.uk are third party vendors that use our marketplace. Additionally, we note that the competitors relevant to our analysis are roughly 250 competitors with more than 20 overlaping products. 

Now let's creat a DF for all competitors and its prices that we can choose to use later:

In [ ]:
# one column for each competitor on our DF:
# This DF takes about an hour to compute. If nothing change, use the pickled file
# saved on the next cells.

number_of_stores = 4137 #4137 is the max! 250 gives the 5th(ish) percentile
comp_names = [i[0] for i in comp_count.most_common()[:number_of_stores]] 

# DF with columns for each of the top competitors
comp_df = pd.DataFrame(columns=comp_names) 

for i in range(10000):
    if type(df.loc[i].sellers) != str: 
        # either a dict or a list of dicts cotaining all companies and prices
        data_obj = df.loc[i].sellers['seller'] 
        to_append={}
        if type(data_obj) == dict: # when sellers >= 2
            comp_name =  list(data_obj.values())[0]
            comp_price = float(list(data_obj.values())[1][1:].replace(',',''))
            if comp_name in comp_df.columns: 
                to_append[comp_name] = comp_price
            
        elif type(data_obj) == list: # when sellers =1
            for rec in data_obj:
                comp_name = list(rec.values())[0]
                comp_price = float(list(rec.values())[1][1:].replace(',',''))
                to_append[comp_name] = comp_price

        comp_df.loc[i] = to_append

    
    else: comp_df.loc[i] = np.nan


In [ ]:
# save the competitor's DF as a serialized file to avoid having to run again.
path = '/content/drive/MyDrive/walmart c1 bootcamp/'

pickled_comp_df = open(path + 'comp_df.pickle', 'wb')
pickle.dump(comp_df, pickled_comp_df)
pickled_comp_df.close()

In [ ]:
# alternatively, run this cell to save as csv (slower, smaller, may lose column types)
comp_df.to_csv(path + 'comp_df.csv', mode = 'wb')

In [ ]:
# retrieve the pickle file from drive
path = '/content/drive/MyDrive/walmart c1 bootcamp/comp_df.pickle'

comp_df = pd.read_pickle(path)
comp_df.shape

(10000, 4137)

In [ ]:
'''Calculates the mean relative price of competitors against our price and
   a confidence metric according to:

     'competitors_avg_price' = mean(competitor_price[i]/our_price - 1)
        i.e. 0 means the competitor has the same price as ours. 
             0.5 means the competitor sells that product by 50% more than us.
             -0.5 means the competitor sells that product by 50 less than us.

     'price_confidence_score' = mean(abs(competitor_price[i]/our_price - 1))
        i.e. 0 means all competitors and us use the same price.
             the higher the score, more variability at our price.
             NOTE: this number does not have a business interpretation.
     '''

# calculate the relative prices for each seller:
comp_df = comp_df.apply(lambda x: x/df['price'] - 1)

# create mean and confidence rows:
comp_df.insert(0, 'competitors_avg_price', comp_df.mean(axis=1))
comp_df.insert(0, 'competitors_price_RMS', comp_df.apply(abs).mean(axis=1))
comp_df.insert(0, 'competitors_count', comp_df.apply(abs).count(axis=1)) 


In [ ]:
# average prices for main 10 competitors
comp_df.loc[:,'OnlineRetail.co.uk':'GamesQuest'].mean()

OnlineRetail.co.uk      0.027651
Buy-For-Less-Online     0.201137
M&E Store-UK            6.749442
M and N Media US        2.630877
MEGA-MAG                2.370197
francejouet             2.125226
Gameseek                0.225771
Sam Store US            5.034409
Gaugemaster            -0.054822
Smaller World Future    1.196180
GamesQuest              0.021767
dtype: float64

In [ ]:
m_and_e_prices = comp_df['M&E Store-UK'].dropna()
m_and_e_prices.sort_values(ascending=False).head()

8020    441.486528
3583     35.527143
2963     34.419540
1090     31.627633
6627     30.858333
Name: M&E Store-UK, dtype: float64

In [ ]:
df.loc[8020].product_name

'Manhattan Toy Groovy Girls Brenna Doll'

In [ ]:
df.loc[8020].sellers

"{'seller': [{'Seller_name_1': 'OnlineRetail.co.uk', 'Seller_price_1': '£12.99'}, {'Seller_name_2': 'LittleImpsToys', 'Seller_price_2': '£13.70'}, {'Seller_name_3': 'Soup Dragon', 'Seller_price_3': '£12.90'}, {'Seller_name_4': 'MEGA-MAG', 'Seller_price_4': '£38.89'}, {'Seller_name_5': 'M&E Store-UK', 'Seller_price_5': '£5,747.90'}]}"

We see there is either an error at the M&E Store price, or missing information. As our final tool will be intended to be sued by marketers that are better equiped to judge this kind of event, we will leave this data as is. Same applies to stores like Sam Store US.

In [ ]:
comp_df.columns

Index(['competitors_count', 'competitors_price_RMS', 'competitors_avg_price',
       'OnlineRetail.co.uk', 'Buy-For-Less-Online', 'M&E Store-UK',
       'M and N Media US', 'MEGA-MAG', 'francejouet', 'Gameseek',
       ...
       'arotherham2', 'stumagu80', 'tedhay', 'RavenEnterprises2016',
       'ofinnegan', 'Side Two', 'blockies', 'Best Deals Direct',
       'The Movie Shop Ltd', 'GiftCompany'],
      dtype='object', length=4140)

In [ ]:
# Investigate how the competitors are within our categories
df[['competitors_count', 
    'competitors_price_RMS', 
    'competitors_avg_price']] = comp_df[['competitors_count', 
                                         'competitors_price_RMS', 
                                         'competitors_avg_price']]
